相较于 CNN 在图像识别和检测方面的广泛应用，**基于序列模型的 RNN 的应用方面则是语音识别、文本翻译和自然语言处理等其他更为激动人心的领域**。所以，正如 CNN 在计算机视觉中的应用一样，在 RNN 中笔者将重点关注其在自然语言处理的应用与研究。
![请添加图片描述](https://img-blog.csdnimg.cn/20200615232011296.png)
**RNN 使用场景**
>  相较于 DNN 和 CNN，RNN 网络结构有什么特别之处？它与前两者又有哪些不一样的结构设计？
>> 在对 RNN 的结构进行深入了解之前，我们必须对使用 RNN 面临的问题场景进行梳理。假设我们在进行语音识别时，**给定了一个输入音频片段 x，要求我们输出一个文本片段 y，其中输入 x 是一个按照时间播放的音频片段，y 是一个按照顺序排列的单词组成的一句话，所以在 RNN 中我们的输入输出都是序列性质的**。针对这样的输入输出（x,y）的有监督学习，最适合的神经网络结构就是循环神经网络。为什么循环神经网络就最适用这种场景？

> **在正式介绍 RNN 前，我们先来看下对于序列问题使用常规的神经网络看看会有什么问题。**

> 假设我们现在需要对输入的一段话识别其中每个单词是否是人名，即输入是一段文本序列，输出是一个每个单词是否是人名的序列。假设这段话有9个单词，我们将其转化为 9 个 one-hot 向量输入到标准神经网络中去，经过一些隐藏层和激活函数得到最终 9 个值为 0/1 的输出。但这样做的问题有两个：

>> 一是输入输出的长度是否相等以及输入大小不固定的问题。在语音识别问题中，输入音频序列和输出文本序列很少情况下是长度相等的，普通网络难以处理这种问题。

>> 二是普通神经网络结构不能共享从文本不同位置上学到的特征，简单来说就是如果神经网络已经从位置 1 学到了 louwill 是一个人名，那么如果 louwill 出现在其他位置，神经网络就可以自动识别到它就是已经学习过的人名，这种共享可以减少训练参数和提高网络效率，普通网络不能达到这样的目的。

所以直观上看，普通神经网络和循环神经网络的区别如下图所示：
![请添加图片描述](https://img-blog.csdnimg.cn/20200615232011283.png)

**RNN 结构**
> 假设我们将一个句子输入 RNN，第一个输入的单词就是 x1, 我们将 x1 输入到神经网络，经过隐藏层得到输出判断其是否为人名，即输出为 y1。同时网络初始化隐藏层激活值，并在隐藏层中结合输入 x1 进行激活计算传入到下一个时间步。当输入第二个单词 x2 的时候，除了使用 x2 预测输出 y2 之外，当前时间步的激活函数会基于上一个时间步的进行激活计算，即第二个时间步利用了第一个时间步的信息。这便是循环（Recurrent）的含义。如此下去，一直到网络在最后一个时间步输出 yn 和 激活值 an。所以在每一个时间步中，RNN 传递一个激活值到下一个时间步中用于计算。
![请添加图片描述](https://img-blog.csdnimg.cn/20200615232011279.png)
> 上图便是循环神经网络的基本结构。左边是一个统一的表现形式，右边则是左边的展开图解。在这样的循环神经网络中，当我们在预测 yt 时，不仅要使用 xt 的信息，还要使用 xt-1 的信息，因为在横轴路径上的隐藏层激活信息得以帮助我们预测 yt。
> 所以， RNN 单元结构通常需要两次计算，**一次是隐藏层隐变量激活函数的计算，一个是结合隐变量和输入的计算。** 一个 RNN 单元和两次计算如下图所示：
![请添加图片描述](https://img-blog.csdnimg.cn/20200615232011298.png)

当多个 RNN 单元组合到一起便是 RNN 结构：
![请添加图片描述](https://img-blog.csdnimg.cn/20200615232011270.png)

##       定义 sigmoid 和 softmax 函数：

In [79]:
import torch  
def softmax(x):
    x=(x-torch.max(x)).exp() 
    return x/x.sum(dim=1,keepdim=True)
def sigmoid(x):
    return 1/1+torch.exp(-x)
x=torch.rand((3,5))
sigmoid(x)

tensor([[1.4647, 1.6273, 1.5951, 1.5470, 1.5285],
        [1.5339, 1.4635, 1.8900, 1.3998, 1.4948],
        [1.6439, 1.7627, 1.5095, 1.5058, 1.8211]])

## 定义 RNN 单元结构：

In [80]:
def rnn_cell_forward(xt,a_prev,params):
    wa=params['wa']
    wx=params['wx']
    wy=params['wy']
    ba=params['ba']
    by=params['by']
    a_next=torch.tanh(torch.matmul(xt,wx)+torch.matmul(a_prev,wa)+ba)
    yt_pred=softmax(torch.matmul(a_next,wy)+by)
    cache=(xt,a_prev,params)
    return a_next,yt_pred,cache
xt=torch.rand((3,6))
a_prev=torch.rand((3,5))
wa=torch.rand((5,5))
wx=torch.rand((6,5)) 
ba=torch.rand((1,5))
wy=torch.rand((5,2))
by=torch.rand((1,2))
params={'wa':wa,'wx':wx,'ba':ba,'wy':wy,'by':by}
a_next,yt_pred,cache=rnn_cell_forward(xt,a_prev,params)
a_next.size() ,yt_pred.size() 
# yt_pred.sum(dim=1,keepdim=True)

(torch.Size([3, 5]), torch.Size([3, 2]))

##    基于 RNN 单元构建 RNN 网络结构：

In [81]:
def rnn_forward(x,a0,params):
    l_x,w_x,n_x=x.shape
    ba,wy=params['ba'],params['wy']
    a_next=a0
    a=torch.zeros((l_x,ba.shape[1],n_x))
    yt=torch.zeros((l_x,wy.shape[1],n_x))
    caches=[[] for _ in range(n_x)]
    for t in range(n_x):
        a_next,yt_pred,cache=rnn_cell_forward(x[...,t],a_next,params)
        a[...,t]=a_next 
        yt[...,t]=yt_pred
        caches[t]+=[cache] 
    caches=(caches,x)
    return a,yt,caches

xt=torch.rand((3,6,7))
a0=torch.rand((3,5))
wa=torch.rand((5,5))
wx=torch.rand((6,5)) 
ba=torch.rand((1,5))
wy=torch.rand((5,2))
by=torch.rand((1,2))
params={'wa':wa,'wx':wx,'ba':ba,'wy':wy,'by':by}     
a,yt,caches=rnn_forward(xt,a0,params)
a.size(),yt.size() 
caches[0][1]

[(tensor([[0.4461, 0.1863, 0.8687, 0.0956, 0.5994, 0.6879],
          [0.1863, 0.4802, 0.8571, 0.8659, 0.7129, 0.5296],
          [0.6390, 0.2908, 0.6040, 0.3276, 0.2229, 0.5827]]),
  tensor([[0.9962, 0.9745, 0.9974, 0.9883, 0.9851],
          [0.9994, 0.9930, 0.9990, 0.9947, 0.9993],
          [0.9988, 0.9904, 0.9977, 0.9874, 0.9947]]),
  {'wa': tensor([[0.1900, 0.7057, 0.5123, 0.4440, 0.1618],
           [0.9847, 0.0175, 0.6940, 0.3380, 0.3042],
           [0.9820, 0.3948, 0.7807, 0.0740, 0.9785],
           [0.0327, 0.6639, 0.0110, 0.0726, 0.3150],
           [0.9289, 0.4353, 0.4073, 0.5214, 0.8053]]),
   'wx': tensor([[0.4280, 0.7754, 0.6107, 0.1731, 0.6296],
           [0.6651, 0.0597, 0.1774, 0.2797, 0.2999],
           [0.2733, 0.4996, 0.7758, 0.5398, 0.1028],
           [0.3013, 0.1577, 0.7483, 0.6863, 0.7298],
           [0.1057, 0.7049, 0.9643, 0.5543, 0.5266],
           [0.9915, 0.0670, 0.2047, 0.0556, 0.4298]]),
   'ba': tensor([[0.8890, 0.2454, 0.5130, 0.7851, 0.4866]]),
